In [2]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from roboflow import Roboflow
import json
import PIL

In [3]:
#Citations: the code below reference code from the online article 
#"Image classification using Support Vector Machine (SVM) in Python"
#url: https://www.geeksforgeeks.org/image-classification-using-support-vector-machine-svm-in-python/

In [4]:
#Citations: this code is given by reboflow to install dataset
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="gMTVjr2tVu8feJmkrnPs")
project = rf.workspace("aps360-project-6tmc3").project("aps360_-2")
dataset = project.version(1).download("coco")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to APS360_(2)-1 in coco:: 100%|████████████████████| 1445/1445 [00:05<00:00, 273.46it/s]


In [5]:
#data preprocessing
def open_coco_file(coco_annots_file_path):
    #takes in the file path to a coco annotations and  
    #returns a dictionary of image id and a tuple of image file path and category
    flat_img_list = []
    labels_list = []
    with open(os.path.join(coco_annots_file_path,"_annotations.coco.json"), 'r') as f:
            file = json.load(f)
    
    for id in range(len(file['images'])):
        img_path = file['images'][id]['file_name']
        img_label = file['annotations'][id]['category_id']
        
        img_arr = np.array(PIL.Image.open(os.path.join(coco_annots_file_path,img_path)))
        flat_img_list.append(resize(img_arr,(150,150,3)).flatten())
        if img_label == 1:
            labels_list.append('seatbelt')
        elif img_label == 2:
            labels_list.append('no-seatbelt')
    df=pd.DataFrame(flat_img_list) 
    df['Labels']=labels_list
    return df
        

In [6]:
train_path = "./APS360_(2)-1/train"
test_path = "./APS360_(2)-1/test"
df_train = open_coco_file(train_path)
df_test = open_coco_file(test_path)

In [7]:
df_test.shape

(272, 67501)

In [8]:
#input data 
x_train=df_train.iloc[:,:-1] 
x_test=df_test.iloc[:,:-1]
#output data
y_train=df_train.iloc[:,-1]
y_test=df_test.iloc[:,-1]
y_train[:4]

0       seatbelt
1       seatbelt
2    no-seatbelt
3       seatbelt
Name: Labels, dtype: object

In [9]:
# Defining the parameters grid for GridSearchCV
param_grid={'C':[0.1,1,10,100],
            'gamma':[0.0001,0.001,0.1,1],
            'kernel':['rbf','poly']}
  
# Creating a support vector classifier
svc=svm.SVC(probability=True)
# Creating a model using GridSearchCV with the parameters grid
model=GridSearchCV(svc,param_grid)  

In [ ]:
# Training the model using the training data
model.fit(x_train,y_train)

In [ ]:
# Testing the model using the testing data
y_pred = model.predict(x_test)
  

In [ ]:
# Calculating the accuracy of the model
accuracy = accuracy_score(y_pred, y_test)
      
# Print the accuracy of the model
print(f"The model is {accuracy*100}% accurate")

In [ ]:
print(classification_report(y_test, y_pred, target_names=['seatbelt', 'no-seatbelt']))